In [1]:
import os
import s3fs
import pandas as pd

os.environ["AWS_ACCESS_KEY_ID"] = 'G1ETFXPKCALLYM3LIIPQ'
os.environ["AWS_SECRET_ACCESS_KEY"] = 'kRUUU3MEqM6+ZBY16+h7cd8xUZtp7agx5kaW7ioj'
os.environ["AWS_SESSION_TOKEN"] = 'eyJhbGciOiJIUzUxMiIsInR5cCI6IkpXVCJ9.eyJhY2Nlc3NLZXkiOiJHMUVURlhQS0NBTExZTTNMSUlQUSIsImFsbG93ZWQtb3JpZ2lucyI6WyIqIl0sImF1ZCI6WyJtaW5pby1kYXRhbm9kZSIsIm9ueXhpYSIsImFjY291bnQiXSwiYXV0aF90aW1lIjoxNzcwOTk4NzM2LCJhenAiOiJvbnl4aWEiLCJlbWFpbCI6Imx1Y2FzLmN1bXVuZWxAZW5zYWUuZnIiLCJlbWFpbF92ZXJpZmllZCI6dHJ1ZSwiZXhwIjoxNzcyMTgyMjA1LCJmYW1pbHlfbmFtZSI6IkN1bXVuZWwiLCJnaXZlbl9uYW1lIjoiTHVjYXMiLCJncm91cHMiOlsiVVNFUl9PTllYSUEiLCJzdGF0YXBwLXNlZ21lZGljIl0sImlhdCI6MTc3MTU3NzQwNSwiaXNzIjoiaHR0cHM6Ly9hdXRoLmxhYi5zc3BjbG91ZC5mci9hdXRoL3JlYWxtcy9zc3BjbG91ZCIsImp0aSI6Im9ucnRydDoyM2E0NWZlNy0yZDFmLTE2NTQtMjExNi04NzBmYmJmMzhjZGQiLCJuYW1lIjoiTHVjYXMgQ3VtdW5lbCIsInBvbGljeSI6InN0c29ubHkiLCJwcmVmZXJyZWRfdXNlcm5hbWUiOiJsYWIiLCJyZWFsbV9hY2Nlc3MiOnsicm9sZXMiOlsib2ZmbGluZV9hY2Nlc3MiLCJ1bWFfYXV0aG9yaXphdGlvbiIsImRlZmF1bHQtcm9sZXMtc3NwY2xvdWQiXX0sInJlc291cmNlX2FjY2VzcyI6eyJhY2NvdW50Ijp7InJvbGVzIjpbIm1hbmFnZS1hY2NvdW50IiwibWFuYWdlLWFjY291bnQtbGlua3MiLCJ2aWV3LXByb2ZpbGUiXX19LCJyb2xlcyI6WyJvZmZsaW5lX2FjY2VzcyIsInVtYV9hdXRob3JpemF0aW9uIiwiZGVmYXVsdC1yb2xlcy1zc3BjbG91ZCJdLCJzY29wZSI6Im9wZW5pZCBwcm9maWxlIGdyb3VwcyBlbWFpbCIsInNpZCI6IjgxNDA0Yzg0LTk2YjAtZGFjMS03NDEzLTBjZmZjMzQ2MWVlZiIsInN1YiI6ImUyZDc4NjRjLTcwMzItNDI0ZC04OTA2LWU0ZjhiNDFjYzAwMyIsInR5cCI6IkJlYXJlciJ9.5vz6y3i0NK0kigDqUhme6q9ToYTpDurBViuffXfR_loD87M3U9V9FnBujdeAShKjIixGY_6tw8NrJ8VTthG36Q'
os.environ["AWS_DEFAULT_REGION"] = 'us-east-1'
fs = s3fs.S3FileSystem(
    client_kwargs={'endpoint_url': 'https://'+'minio.lab.sspcloud.fr'},
    key = os.environ["AWS_ACCESS_KEY_ID"], 
    secret = os.environ["AWS_SECRET_ACCESS_KEY"], 
    token = os.environ["AWS_SESSION_TOKEN"])

with fs.open("s3://lab/art_net_dec_cite.parquet") as f:
    jdf = pd.read_parquet(f)

In [18]:
with fs.open("s3://lab/art_net_dec_cite.parquet", "wb") as f:
    df.to_parquet(f)

In [13]:
import pandas as pd
import numpy as np
import json

def build_author_coauthor(df, authors_col="authors"):
    rows = []

    for _, row in df.iterrows():
        authors = row[authors_col]

        # convertir JSON string
        if isinstance(authors, str):
            try:
                authors = json.loads(authors)
            except:
                continue

        # convertir numpy array -> list
        if isinstance(authors, np.ndarray):
            authors = authors.tolist()

        if not isinstance(authors, list):
            continue

        names = [
            a.get("name")
            for a in authors
            if isinstance(a, dict) and a.get("name")
        ]

        for author in names:
            coauthors = [a for a in names if a != author]

            rows.append({
                "author": author,
                "coauthors": coauthors,
                "year" : row["year"],
                "citations": row["citations"]
            })

    return pd.DataFrame(rows)


In [14]:
df = build_author_coauthor(jdf)

In [4]:
#more than 10 pub
df = df[df["author"].map(df["author"].value_counts()) > 10]

In [20]:
import pandas as pd
from collections import defaultdict

def permanent_2hop_loss_light(df):
    """
    For each author and year, compute the number of 2-hop coauthors 
    who never appear again in the ego's network after that year.
    Only returns rows where at least one person leaves the ego's network.

    Parameters
    ----------
    df : pd.DataFrame
        Columns: ['author', 'coauthors', 'year']
        'coauthors' must be a list of author names

    Returns
    -------
    pd.DataFrame
        Columns: ['author', 'year', 'lost_2hop', 'total_2hop', 'lost_ratio']
        Only rows where lost_2hop > 0
    """

    df = df.copy()
    df["year"] = pd.to_numeric(df["year"], errors="coerce")
    df = df.dropna(subset=["year"])
    df["year"] = df["year"].astype(int)
    df = df[df["year"] < 2009]

    # Step 1: Build cumulative coauthor graph year by year
    neighbors = defaultdict(set)
    years = sorted(df["year"].unique())
    ego_year_co2 = defaultdict(dict)  # ego_year_co2[author][year] = co2 set at that year

    for Y in years:
        df_current = df[df["year"] == Y]

        # Update neighbors with new publications
        for row in df_current.itertuples(index=False):
            author = row.author
            coauthors = row.coauthors
            if not isinstance(coauthors, list):
                continue

            for c in coauthors:
                neighbors[author].add(c)
                neighbors[c].add(author)

        # Compute 2-hop sets for each active author this year
        active_authors = set(df_current["author"])
        for author in active_authors:
            co1 = neighbors[author]
            co2 = set()
            for c in co1:
                co2.update(neighbors[c])
            co2.discard(author)
            co2 -= co1
            ego_year_co2[author][Y] = co2

    # Step 2: Backward pass to count permanent 2-hop loss
    results = []

    for author, year_co2 in ego_year_co2.items():
        future_seen = set()  # all 2-hop nodes that appear after current year
        for Y in sorted(year_co2.keys(), reverse=True):
            co2 = year_co2[Y]
            lost_2hop = co2 - future_seen
            total_2hop = len(co2)
            ratio = len(lost_2hop) / total_2hop if total_2hop > 0 else 0

            # Only keep rows where at least one person leaves ego's network
            if len(lost_2hop) > 0:
                results.append({
                    "author": author,
                    "year": Y,
                    "lost_2hop": len(lost_2hop),
                    "total_2hop": total_2hop,
                    "lost_ratio": ratio
                })

            future_seen.update(co2)  # update future seen nodes

    return pd.DataFrame(results)

In [21]:
result = permanent_2hop_loss_light(df)

In [24]:
result["lost_2hop"].value_counts()

lost_2hop
1       67583
2       32265
3       22374
4       17879
5       15038
        ...  
1557        1
1162        1
925         1
844         1
1220        1
Name: count, Length: 1316, dtype: int64

In [40]:
with fs.open("s3://lab/name_2hop_5yinac.parquet", "wb") as f:
    result.to_parquet(f)

In [30]:
#df["year"] = pd.to_numeric(df["year"], errors="coerce").astype(int)
merge = df.merge(result, how="left", on=["author","year"])#[result["inactive_coauthors_2hop"]!=0]

In [34]:
merge[["lost_2hop","total_2hop","lost_ratio"]] = merge[["lost_2hop","total_2hop","lost_ratio"]].fillna(0)
merge = merge.dropna()
merge

,author,coauthors,year,citations,lost_2hop,total_2hop,lost_ratio
8,Fridolin H. Heidler,[Wolfgang Schulz],2022,0.0,0.0,0.0,0.0
9,Wolfgang Schulz,[Fridolin H. Heidler],2022,0.0,0.0,0.0,0.0
12,Stefan Radel,"[Johannes Schnöller, Ana Dominguez-Vidal, Mart...",2008,9.0,0.0,0.0,0.0
13,Johannes Schnöller,"[Stefan Radel, Ana Dominguez-Vidal, Martin Grö...",2008,9.0,0.0,0.0,0.0
14,Ana Dominguez-Vidal,"[Stefan Radel, Johannes Schnöller, Martin Grös...",2008,9.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
9634011,Gayane Grigoryan,"[Oliver Gronz, Oliver Gutjahr, Rita Ley, Andre...",2012,25.0,0.0,0.0,0.0
9634012,Oliver Gronz,"[Gayane Grigoryan, Oliver Gutjahr, Rita Ley, A...",2012,25.0,0.0,0.0,0.0
9634013,Oliver Gutjahr,"[Gayane Grigoryan, Oliver Gronz, Rita Ley, And...",2012,25.0,0.0,0.0,0.0
9634014,Rita Ley,"[Gayane Grigoryan, Oliver Gronz, Oliver Gutjah...",2012,25.0,0.0,0.0,0.0


In [38]:
merge[merge["lost_2hop"]<20]["citations"].corr(merge[merge["lost_2hop"]<20]["lost_ratio"])

np.float64(0.008974998468397)

In [37]:
merge[merge["lost_2hop"]>50]

,author,coauthors,year,citations,lost_2hop,total_2hop,lost_ratio
34,Dirk Hammerschmidt,"[Thomas Bretterklieber, Gert Holler, Daniel Hr...",2008,4.0,59.0,59.0,1.0
353,Walter Brenner,"[Rüdiger Zarnekow, Jörn Kruse, Andreas Sidler]",2008,2.0,170.0,170.0,1.0
382,Andreas Poppe,[],2007,0.0,52.0,52.0,1.0
444,Helmut Malleck,[Lothar Fickert],2008,1.0,60.0,60.0,1.0
610,M. Omair Shafiq,"[Dieter Fensel, Reto Krummenacher, Eva Kühn, J...",2007,15.0,138.0,138.0,1.0
...,...,...,...,...,...,...,...
9633529,Naveen Srinivasan,"[David L. Martin 0001, Mark H. Burstein, Drew ...",2007,321.0,234.0,234.0,1.0
9633530,Daniel A. Menascé,[Vasudeva Akula],2007,1.0,576.0,576.0,1.0
9633537,Hai He,"[Weiyi Meng, Yiyao Lu, Clement T. Yu, Zonghuan...",2007,35.0,88.0,88.0,1.0
9633539,Yiyao Lu,"[Hai He, Weiyi Meng, Clement T. Yu, Zonghuan Wu]",2007,35.0,87.0,87.0,1.0


In [25]:
df["year"] = df["year"].astype(int)
result = result.merge(df[df["year"]<2009][["coauthors","author"]], how="left", on="author")
result = result[~result["coauthors"].isna()]

In [31]:
result=result.explode(["coauthors"])

In [39]:

import networkx as nx

G = nx.from_pandas_edgelist(result.dropna(),"author","coauthors")

In [43]:
betweenness = nx.betweenness_centrality(G, normalized=True, k=100)
result["cent"] = result["author"].map(betweenness)

In [45]:
#result["cent"] = result["author"].map(betweenness)
result[result["cent"]!=0]

,author,year,inactive_coauthors_2hop,total_2hop,inactive_ratio,coauthors,cent
0,Willard Van Orman Quine,1936,0,0,0.0,NaN,0.000006
1,Willard Van Orman Quine,1936,0,0,0.0,NaN,0.000006
2,Willard Van Orman Quine,1936,0,0,0.0,William Craig,0.000006
3,Willard Van Orman Quine,1936,0,0,0.0,NaN,0.000006
4,Willard Van Orman Quine,1936,0,0,0.0,NaN,0.000006
...,...,...,...,...,...,...,...
16528079,Ying Jing,2008,0,3,0.0,Minghui Wu 0001,0.000002
16528079,Ying Jing,2008,0,3,0.0,Canghong Jin,0.000002
16528093,Simone Campanoni,2008,0,0,0.0,NaN,NaN
16528095,José-Luis Sánchez-Romero,2008,0,0,0.0,NaN,NaN


In [50]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.formula.api as smf

result=result.dropna()
model=smf.ols(formula='cent ~ inactive_ratio', data=result).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   cent   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     4219.
Date:                Fri, 13 Feb 2026   Prob (F-statistic):               0.00
Time:                        20:30:33   Log-Likelihood:             1.2302e+08
No. Observations:            22936382   AIC:                        -2.460e+08
Df Residuals:                22936380   BIC:                        -2.460e+08
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          0.0004   2.54e-07   1671.202      0.000       0.000       0.000
inactive_ratio    -0.0005   7.26e-06    -64.950      0.000      -0.000      -0.000
==============================================================================
Omnibus:                 26253803.472   Durbin-Watson:                   0.019
Prob(Omnibus):                  0.000   Jarque-Bera (JB):       2364950794.661
Skew:                           6.125   Prob(JB):                         0.00
Kurtosis:                      51.213   Cond. No.                         30.7
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""